In [34]:
from sqlalchemy import create_engine, inspect, MetaData, Table, insert, func, select, delete, or_, and_
import pandas as pd
pd.set_option('display.width',None)
pd.set_option('display.max_columns',None) 
import requests
import logging
from datetime import datetime

logging.basicConfig(level=logging.ERROR)

In [11]:
engine = create_engine('sqlite:///olympicsdata.sqlite')
metadata = MetaData()
metadata.reflect(bind=engine)
print(metadata.tables)

FacadeDict({'athletes': Table('athletes', MetaData(), Column('id', INTEGER(), table=<athletes>), Column('name', VARCHAR(), table=<athletes>), Column('gender', VARCHAR(), table=<athletes>), Column('age', INTEGER(), table=<athletes>), Column('height', INTEGER(), table=<athletes>), Column('weigth', INTEGER(), table=<athletes>), schema=None), 'countries': Table('countries', MetaData(), Column('id', INTEGER(), table=<countries>), Column('country', VARCHAR(), table=<countries>), Column('region', VARCHAR(), table=<countries>), schema=None), 'wintergames': Table('wintergames', MetaData(), Column('sport', VARCHAR(), table=<wintergames>), Column('event', VARCHAR(), table=<wintergames>), Column('year', DATE(), table=<wintergames>), Column('athlete_id', INTEGER(), table=<wintergames>), Column('country_id', INTEGER(), table=<wintergames>), Column('medal', VARCHAR(), table=<wintergames>), schema=None)})


In [12]:
print(inspect(engine).get_table_names())

['athletes', 'countries', 'wintergames']


In [13]:
wintergames = metadata.tables.get('wintergames')
athletes = metadata.tables.get('athletes')
countries = metadata.tables.get('countries')

In [15]:
# Check how many distinct values in medal column of table wintergames exists
with engine.connect() as conn:
    stmt = select([wintergames.columns.medal.distinct()])
    results = conn.execute(stmt)
    distinct_values = [row[0] for row in results]
    print("Distinct values in medal column are following: ", distinct_values)

Distinct values in medal column are following:  ['', 'Bronze', 'Gold', 'Silver']


In [27]:
# Find top 5 countries which won the most Gold medals historically
with engine.connect() as conn:
    stmt = select([countries.c.country,func.count().label('gold_medal_count')]) \
            .select_from(wintergames.join(countries, wintergames.c.country_id == countries.c.id)) \
            .where(wintergames.c.medal == 'Gold') \
            .group_by(countries.c.country) \
            .order_by(func.count().desc()) \
            .limit(5)
    results = conn.execute(stmt).fetchall()
    for row in results:
          print(f"Country: {row['country']}, gold medals won: {row['gold_medal_count']}")

Country: NOR - Norway, gold medals won: 13
Country: SWE - Sweden, gold medals won: 8
Country: RUS - Russia, gold medals won: 5
Country: UKR - Ukraine, gold medals won: 4
Country: SUI - Switzerland, gold medals won: 4


In [31]:
# Find top male athlete with most won medals (gold, silver or bronze)
with engine.connect() as conn:
    stmt = select([athletes.c.name,func.count().label('total_medals_won')]) \
            .select_from(wintergames.join(athletes,wintergames.c.athlete_id == athletes.c.id)) \
            .where(and_(athletes.c.gender == 'M',wintergames.c.medal.in_(['Gold','Silver','Bronze']))) \
            .group_by(athletes.c.name) \
            .order_by(func.count().desc())
    topmale = conn.execute(stmt).first()
    print(f"Best male athlete {topmale['name']} has won {topmale['total_medals_won']} medals")

Best male athlete Ondej Moravec has won 3 medals


In [36]:
# Find how many countries earned at least one medal
with engine.connect() as conn:
    stmt = select([func.count(wintergames.c.country_id.distinct())]) \
            .where(wintergames.c.medal.notin_(['']))
    nbcountries = conn.execute(stmt).scalar()
    print(f"Total number of countries which won at least 1 medal: {nbcountries}")

Total number of countries which won at least 1 medal: 18


In [46]:
# Percentage of medals won by men and women in database
with engine.connect() as conn:
    gender_stmt = select([athletes.c.gender,func.count(wintergames.c.medal).label('total_medals_won')]) \
            .select_from(wintergames.join(athletes, wintergames.c.athlete_id == athletes.c.id)) \
            .where(wintergames.c.medal.notin_([''])) \
            .group_by(athletes.c.gender)
    resultsgender = conn.execute(gender_stmt).fetchall()
    total_stmt = select([func.count(wintergames.c.medal).label('all_count')]).where(wintergames.c.medal.notin_(['']))
    totalmedals = conn.execute(total_stmt).scalar()
    for r in results:
        gender = r[0]
        medals_won = r[1]
        percentage = (medals_won / totalmedals * 100)
        print(f"{gender}: {medals_won} percentage: {percentage:.2f}%")

F: 75 percentage: 49.67%
M: 76 percentage: 50.33%
